# Import repo

In [2]:
!git clone https://github.com/FernandoDoreto/convert-streamlit-to-django.git

fatal: destination path 'convert-streamlit-to-django' already exists and is not an empty directory.


In [3]:
%cd /content/convert-streamlit-to-django

/content/convert-streamlit-to-django


In [4]:
pwd

'/content/convert-streamlit-to-django'

# Load dataset

In [4]:
pip install streamlit

In [5]:
from src.processing.data_management import LoadIrisDataset

In [6]:
df = LoadIrisDataset()
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


* Train Test split

In [7]:
from config import config
from src.processing.data_management import TrainTestSplit

In [8]:
 X_train, X_test,y_train, y_test = TrainTestSplit(df=df,TARGET=config.ClfIrisSpecies_TARGET)

# Grid Search on 1 pipeline

In [59]:
from config import config
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import GridSearchCV

ClfIrisSpecies_DT = Pipeline(
    [       
        ("feat_selection",SelectFromModel(DecisionTreeClassifier(random_state=config.RANDOM_STATE))),
        ("feat_scaling",StandardScaler()),
        ("model", DecisionTreeClassifier(random_state=config.RANDOM_STATE))
    ]
)


_parameters = {
    'model__splitter': ["best","random"],
    'model__max_depth': [None,3,5,10],
    'model__criterion': ["gini", "entropy"]
}


_pipe = GridSearchCV(
		estimator = ClfIrisSpecies_DT,
		param_grid = _parameters, 
		cv=5,n_jobs=-2,verbose=2)

In [60]:
_pipe.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=None, model__

[Parallel(n_jobs=-2)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=random, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=10, mo

[Parallel(n_jobs=-2)]: Done  80 out of  80 | elapsed:    0.5s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('feat_selection',
                                        SelectFromModel(estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                                                         class_weight=None,
                                                                                         criterion='gini',
                                                                                         max_depth=None,
                                                                                         max_features=None,
                                                                                         max_leaf_nodes=None,
                                                                                         min_impurity_decrease=0.0,
                                                                                         min_impurity_s

In [98]:
PipelineToDeploy = _pipe.best_estimator_
PipelineToDeploy

Pipeline(memory=None,
         steps=[('feat_selection',
                 SelectFromModel(estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                                  class_weight=None,
                                                                  criterion='gini',
                                                                  max_depth=None,
                                                                  max_features=None,
                                                                  max_leaf_nodes=None,
                                                                  min_impurity_decrease=0.0,
                                                                  min_impurity_split=None,
                                                                  min_samples_leaf=1,
                                                                  min_samples_split=2,
                                                                  min_weight_fract

In [70]:
_pipe.best_params_

{'model__criterion': 'gini',
 'model__max_depth': None,
 'model__splitter': 'best'}

In [99]:
# main features in this pipeline -taken from feat selection step
X_train.columns[PipelineToDeploy['feat_selection'].get_support()].to_list()

['petal width (cm)']

In [104]:
from sklearn.metrics import classification_report
print( classification_report(y_test, PipelineToDeploy.predict(X_test)) )

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.83      1.00      0.91        10
           2       1.00      0.80      0.89        10

    accuracy                           0.93        30
   macro avg       0.94      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30



# Grid Search on Stacking pipeline

In [112]:
from config import config
from sklearn.pipeline import Pipeline

from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

# from mlxtend.classifier import StackingClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import StackingClassifier

In [113]:
ClfIrisSpecies_DT = Pipeline(
    [       
        ("feat_selection",SelectFromModel(DecisionTreeClassifier(random_state=config.RANDOM_STATE))),
        ("feat_scaling",StandardScaler()),
        ("model", DecisionTreeClassifier(random_state=config.RANDOM_STATE))
    ]
)


_parameters_DT = {
    'model__splitter': ["best","random"],
    'model__max_depth': [None,3,10],
    'model__criterion': ["gini", "entropy"]
}


PipelineClfIrisSpecies_DT = GridSearchCV(
		estimator = ClfIrisSpecies_DT,
		param_grid = _parameters_DT, 
		cv = 5,n_jobs = -2,verbose = 2,refit=True)



In [114]:
ClfIrisSpecies_RF = Pipeline(
    [       
        ("feat_selection",SelectFromModel(RandomForestClassifier(random_state=config.RANDOM_STATE))),
        ("feat_scaling",StandardScaler()),
        ("model", RandomForestClassifier(random_state=config.RANDOM_STATE))
    ]
)

_parameters_RF = {
    'model__n_estimators': [100,50],
    'model__max_depth': [None,5,10],
    'model__criterion': ["gini", "entropy"]
}

PipelineClfIrisSpecies_RF = GridSearchCV(
		estimator = ClfIrisSpecies_RF,
		param_grid = _parameters_RF, 
		cv = 5,n_jobs = -2,verbose = 2,refit=True)



In [115]:
ClfIrisSpecies_GB = Pipeline(
    [       
        ("feat_selection",SelectFromModel(GradientBoostingClassifier(random_state=config.RANDOM_STATE))),
        ("feat_scaling",StandardScaler()),
        ("model", GradientBoostingClassifier(random_state=config.RANDOM_STATE,learning_rate=0.1))
    ]
)

_parameters_GB = {
    'model__learning_rate': [1e-1,1e-2],
    'model__n_estimators': [100,50]
}


PipelineClfIrisSpecies_GB = GridSearchCV(
		estimator = ClfIrisSpecies_GB,
		param_grid = _parameters_GB, 
		cv = 5,n_jobs = -2,verbose = 2,refit=True)



In [116]:
estimators = [
            ("GB",PipelineClfIrisSpecies_GB),
            ("RF",PipelineClfIrisSpecies_RF),
            ("DT",PipelineClfIrisSpecies_DT)
        ]

ClfIrisStacking = StackingClassifier(
    estimators=estimators,
    # use_probas=True,
    final_estimator=LogisticRegression(solver='liblinear')
)


In [117]:
ClfIrisStacking.fit(X_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] model__learning_rate=0.1, model__n_estimators=100 ...............


[Parallel(n_jobs=-2)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  model__learning_rate=0.1, model__n_estimators=100, total=   0.3s
[CV] model__learning_rate=0.1, model__n_estimators=100 ...............


[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV]  model__learning_rate=0.1, model__n_estimators=100, total=   0.3s
[CV] model__learning_rate=0.1, model__n_estimators=100 ...............
[CV]  model__learning_rate=0.1, model__n_estimators=100, total=   0.3s
[CV] model__learning_rate=0.1, model__n_estimators=100 ...............
[CV]  model__learning_rate=0.1, model__n_estimators=100, total=   0.3s
[CV] model__learning_rate=0.1, model__n_estimators=100 ...............
[CV]  model__learning_rate=0.1, model__n_estimators=100, total=   0.3s
[CV] model__learning_rate=0.1, model__n_estimators=50 ................
[CV] . model__learning_rate=0.1, model__n_estimators=50, total=   0.3s
[CV] model__learning_rate=0.1, model__n_estimators=50 ................
[CV] . model__learning_rate=0.1, model__n_estimators=50, total=   0.3s
[CV] model__learning_rate=0.1, model__n_estimators=50 ................
[CV] . model__learning_rate=0.1, model__n_estimators=50, total=   0.3s
[CV] model__learning_rate=0.1, model__n_estimators=50 ................
[CV] .

...

[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=random, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=random, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=random, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=random 
[CV]  model__criteri

[Parallel(n_jobs=-2)]: Done  60 out of  60 | elapsed:    0.4s finished
[Parallel(n_jobs=-2)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=random, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=random, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=random, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=random, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=random 
[CV]  model__cri

[Parallel(n_jobs=-2)]: Done  60 out of  60 | elapsed:    0.4s finished


StackingClassifier(cv=None,
                   estimators=[('GB',
                                GridSearchCV(cv=5, error_score=nan,
                                             estimator=Pipeline(memory=None,
                                                                steps=[('feat_selection',
                                                                        SelectFromModel(estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                                                                                             criterion='friedman_mse',
                                                                                                                             init=None,
                                                                                                                             learning_rate=0.1,
                                                                                                                             los

In [118]:
from sklearn.metrics import classification_report
print( classification_report(y_test, ClfIrisStacking.predict(X_test)) )

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.83      1.00      0.91        10
           2       1.00      0.80      0.89        10

    accuracy                           0.93        30
   macro avg       0.94      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30

